In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 512
channel = 96
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [6]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
eeg_dataset, splits_path, splits_60, splits_shuffled_path = os.listdir(torch_models_dir)

In [7]:
eeg_dataset = os.path.join(torch_models_dir, eeg_dataset)
splits_path = os.path.join(torch_models_dir, splits_path)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_path)

/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth 
 /media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth


In [10]:
opt = {
    # Dataset options
    "iv": "image",
    "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "run": "none",
    "eeg_dataset": eeg_dataset,
    "splits_path": splits_path,
    "fold": 5,
    #Training options
    "batch_size": 128, #change batch size from 16 to 128
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"incremental",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.iv)

image


In [22]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from data_loader import EEGDataset, Splitter, SplitterWithData
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
def load_dataset(iv,
             offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-4) - 5 fold cross validation
             total,
             classes,
             classifier,
             batch_size,
             GPUindex,
             length, # 512
             channel, # 96
             min_CNN,
             opt,
             kind):        
    # Load dataset
    dataset = EEGDataset(iv, eeg_dataset, classifier, map_idx = None)
    print("DONE: LOAD DATASET")
    # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
    if kind=="from-scratch":
        relabel = False
    if kind=="incremental":
        relabel = False
    if kind=="no-model-file":
        relabel = True
    splitter = {split: SplitterWithData(iv,
                    dataset,
                    splits_path,
                    classes,
                    split_num,
                    split,
                    relabel) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [14]:
# Stacked BiLSTM network
opt.classifier = "Stacked_BiLSTM"
opt.batch_size = 128 # change batch size from 16 to 128
opt.kind = "from-scratch"
opt.run = "imagenet40-1000"
opt.fold = 5

In [15]:
dataset, loaders, splitter = load_dataset(opt.iv,
                             opt.offset,
                             opt.eeg_dataset,
                             opt.splits_path,
                             0, #split_num
                             n_classes,
                             classes,
                             opt.classifier,
                             opt.batch_size,
                             opt.GPUindex,
                             length,
                             channel,
                             min_CNN,
                             opt,
                             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [16]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 32000 idx, val: 4000 idx, test: 4000 idx}
# Each loader batch has {train: 2000 idx, val: 250idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i in range(0, 40):
    eeg, label_val = splitter["val"][i*100]
    eeg, label_train = splitter["train"][i*800]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")
# print(splitter["val"].split_idx[:10])

<class 'data_loader.EEGDataset'>
<class 'dict'>
3 [250, 32, 32]
1: Label val: 0; label train: 0
2: Label val: 17; label train: 17
3: Label val: 28; label train: 28
4: Label val: 7; label train: 7
5: Label val: 33; label train: 33
6: Label val: 12; label train: 12
7: Label val: 21; label train: 21
8: Label val: 3; label train: 3
9: Label val: 25; label train: 25
10: Label val: 36; label train: 36
11: Label val: 10; label train: 10
12: Label val: 15; label train: 15
13: Label val: 19; label train: 19
14: Label val: 31; label train: 31
15: Label val: 23; label train: 23
16: Label val: 5; label train: 5
17: Label val: 38; label train: 38
18: Label val: 8; label train: 8
19: Label val: 1; label train: 1
20: Label val: 34; label train: 34
21: Label val: 29; label train: 29
22: Label val: 26; label train: 26
23: Label val: 13; label train: 13
24: Label val: 11; label train: 11
25: Label val: 22; label train: 22
26: Label val: 18; label train: 18
27: Label val: 6; label train: 6
28: Label val:

In [17]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_Stacked_BiLSTM(
  (stacked_bilstm): LSTM(96, 128, num_layers=4, batch_first=True, bidirectional=True)
  (output1): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_Stacked_BiLSTM                [1, 40]                   --
├─LSTM: 1-1                              [1, 512, 256]             1,417,216
├─Linear: 1-2                            [1, 128]                  32,896
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 40]                   5,160
Total params: 1,455,272
Trainable params: 1,455,272
Non-trainable params: 0
Total mult-adds (M): 725.65
Input size (MB): 0.20
Forward/backward pass size (MB): 1.05
Params size (MB): 5.82
Estimated Total Size (MB): 7.07

In [19]:
model_path = (opt.iv+
                  "-"+
                  opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel)+
                  "-"+
                  str(0)) + "-batchsize_128"
                  
channel_idx=None
print(model_path)

image-Stacked_BiLSTM-512-96-0-batchsize_128


In [20]:
print(opt)

Namespace(iv='image', offset=None, results_file='results.pkl', subject=0, run='imagenet40-1000', eeg_dataset='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth', splits_path='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth', fold=5, batch_size=128, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='Stacked_BiLSTM')


In [23]:
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

epoch 1
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=3.691070556640625; accuracy=0.025624999776482582
Batch 200: Loss=3.6891801357269287; accuracy=0.025273436680436134
Validation accuracy: 0.024250000715255737
epoch 2
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=3.6926348209381104; accuracy=0.027109375223517418
Batch 200: Loss=3.6937155723571777; accuracy=0.026679687201976776
Validation accuracy: 0.026750002056360245
epoch 3
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=3.681079626083374; accuracy=0.029921874403953552
Batch 200: Loss=3.6818535327911377; accuracy=0.02812499925494194
Validation accuracy: 0.026750002056360245
epoch 4
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  tor

Validation accuracy: 0.024250000715255737
epoch 29
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=1.2758233547210693; accuracy=0.5600780844688416
Batch 200: Loss=1.3522653579711914; accuracy=0.5475780963897705
Validation accuracy: 0.024000000208616257
epoch 30
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=1.4837334156036377; accuracy=0.5702343583106995
Batch 200: Loss=1.4488565921783447; accuracy=0.5702343583106995
Validation accuracy: 0.02525000087916851
epoch 31
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=1.3108755350112915; accuracy=0.6219531297683716
Batch 200: Loss=1.3154041767120361; accuracy=0.6087109446525574
Validation accuracy: 0.02800000086426735
epoch 32
Size of input:  torch.Size([128, 96, 512])
Size of target:  tor

Batch 200: Loss=0.2763209044933319; accuracy=0.8930078148841858
Validation accuracy: 0.02550000138580799
epoch 57
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=0.37799519300460815; accuracy=0.89453125
Batch 200: Loss=0.4284789264202118; accuracy=0.89124995470047
Validation accuracy: 0.02550000138580799
epoch 58
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=0.3315412998199463; accuracy=0.9056249856948853
Batch 200: Loss=0.4709083139896393; accuracy=0.8986718654632568
Validation accuracy: 0.02850000187754631
epoch 59
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=0.25665083527565; accuracy=0.9154687523841858
Batch 200: Loss=0.441143661737442; accuracy=0.9080859422683716
Validation accuracy: 0.026500001549720764
epoch 60
Size of inpu

Batch 100: Loss=0.16077712178230286; accuracy=0.9222655892372131
Batch 200: Loss=0.33668431639671326; accuracy=0.9206249713897705
Validation accuracy: 0.030750000849366188
epoch 85
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=0.22672297060489655; accuracy=0.9313281178474426
Batch 200: Loss=0.3294481337070465; accuracy=0.9296483993530273
Validation accuracy: 0.02575000189244747
epoch 86
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=0.15627188980579376; accuracy=0.9307812452316284
Batch 200: Loss=0.1843087077140808; accuracy=0.9342578053474426
Validation accuracy: 0.027250001206994057
epoch 87
Size of input:  torch.Size([128, 96, 512])
Size of target:  torch.Size([128])
Size of output:  torch.Size([128, 40])
Batch 100: Loss=0.15738339722156525; accuracy=0.943359375
Batch 200: Loss=0.21475546061992645; accuracy=0.939

In [19]:
val =accuracy_val
test = accuracy_test

print("Validation accuracy: ", val)
print("Test accuracy: ", test)

Validation accuracy:  0.024000000208616257
Test accuracy:  0


In [ ]:
print(accuracy_val)